# Silent Hunter v6.0 - ISS Metagenome Novel Protein Discovery

**100% Data Publication Pipeline**

This notebook processes the complete ISS metagenome dataset (GLDS-69, SRR6356483) to discover novel proteins with publication-ready methodology.

## Requirements
- Google Colab Pro (recommended for 100GB+ disk space)
- ~7-10 hours total runtime
- Google Drive for persistent storage

## Cell 0: Setup and Mount Drive

In [ ]:
# Mount Google Drive and setup logging
from google.colab import drive
import datetime
import hashlib
import os
import json

drive.mount('/content/drive')

# Create directory structure
BASE_DIR = '/content/drive/MyDrive/SilentHunter_v6'
dirs = ['data', 'intermediate', 'databases', 'output', 'audit']
for d in dirs:
    os.makedirs(f'{BASE_DIR}/{d}', exist_ok=True)

# Configuration
SRA_ACCESSION = 'SRR6356483'
THREADS = 4

# Audit logging
AUDIT_LOG = f'{BASE_DIR}/audit/commands.log'

def log_command(cmd, output=""):
    with open(AUDIT_LOG, 'a') as f:
        f.write(f"\n{'='*60}\n")
        f.write(f"TIME: {datetime.datetime.now().isoformat()}\n")
        f.write(f"CMD: {cmd}\n")
        if output:
            f.write(f"OUTPUT: {output[:500]}...\n")
    print(f"[LOGGED] {cmd[:50]}...")

log_command("Session started")
print(f"Base directory: {BASE_DIR}")
print("Audit logging enabled")

## Step 1: Download Full Dataset

In [ ]:
# Install SRA Toolkit
!pip install -q sra-tools

# Download 100% data (NO downsampling)
cmd = f"fastq-dump --split-files --gzip {SRA_ACCESSION}"
!{cmd}
log_command(cmd)

# Verify download
!ls -lh {SRA_ACCESSION}*.fastq.gz

# Count reads
!echo "Read count:" && zcat {SRA_ACCESSION}_1.fastq.gz | wc -l | awk '{{print $1/4 " reads"}}'

# Calculate checksums
!md5sum {SRA_ACCESSION}*.fastq.gz > {BASE_DIR}/audit/raw_data.md5
!cat {BASE_DIR}/audit/raw_data.md5

## Step 2: Quality Control (fastp)

In [ ]:
# Install fastp
!apt-get install -qq fastp

# Run fastp
cmd = f"""fastp \
    -i {SRA_ACCESSION}_1.fastq.gz \
    -I {SRA_ACCESSION}_2.fastq.gz \
    -o clean_1.fastq.gz \
    -O clean_2.fastq.gz \
    --detect_adapter_for_pe \
    --cut_front --cut_tail \
    --cut_window_size 4 \
    --cut_mean_quality 20 \
    --length_required 50 \
    --html {BASE_DIR}/audit/fastp_report.html \
    --json {BASE_DIR}/audit/fastp_report.json \
    --thread {THREADS}"""

!{cmd}
log_command(cmd)

# Display QC stats
with open(f'{BASE_DIR}/audit/fastp_report.json') as f:
    data = json.load(f)
    before = data['summary']['before_filtering']
    after = data['summary']['after_filtering']
    print(f"\nQC Statistics:")
    print(f"Reads before: {before['total_reads']:,}")
    print(f"Reads after: {after['total_reads']:,}")
    print(f"Survival rate: {after['total_reads']/before['total_reads']*100:.1f}%")

## Step 3: Assembly (MEGAHIT)

In [ ]:
# Install MEGAHIT
!wget -q https://github.com/voutcn/megahit/releases/download/v1.2.9/MEGAHIT-1.2.9-Linux-x86_64-static.tar.gz
!tar -xzf MEGAHIT-1.2.9-Linux-x86_64-static.tar.gz
!cp MEGAHIT-1.2.9-Linux-x86_64-static/bin/megahit* /usr/local/bin/

# Run MEGAHIT
cmd = f"""megahit \
    -1 clean_1.fastq.gz \
    -2 clean_2.fastq.gz \
    -o assembly \
    --min-contig-len 500 \
    --k-min 21 \
    --k-max 141 \
    --k-step 12 \
    -t {THREADS} \
    -m 0.9"""

!{cmd}
log_command(cmd)

In [ ]:
# Calculate assembly statistics
seqs = []
with open('assembly/final.contigs.fa') as f:
    s = ''
    for line in f:
        if line.startswith('>'):
            if s: seqs.append(len(s))
            s = ''
        else: s += line.strip()
    if s: seqs.append(len(s))

seqs.sort(reverse=True)
total = sum(seqs)

# N50
n50_sum = 0
for l in seqs:
    n50_sum += l
    if n50_sum >= total/2:
        print(f'N50: {l:,} bp')
        break

print(f'Total contigs: {len(seqs):,}')
print(f'Total assembly size: {total:,} bp')
print(f'Largest contig: {seqs[0]:,} bp')

# Save to Drive
!cp -r assembly {BASE_DIR}/intermediate/

## Step 4: ORF Prediction (Prodigal)

In [ ]:
# Install Prodigal
!apt-get install -qq prodigal

# Run Prodigal in metagenomic mode
cmd = """prodigal \
    -i assembly/final.contigs.fa \
    -a proteins.faa \
    -d genes.fna \
    -o genes.gff \
    -f gff \
    -p meta \
    -q"""

!{cmd}
log_command(cmd)

# Count ORFs
!echo "Total proteins:" && grep -c ">" proteins.faa

# Save to Drive
!mkdir -p {BASE_DIR}/intermediate/orfs
!cp proteins.faa genes.fna genes.gff {BASE_DIR}/intermediate/orfs/

## Step 5A: Copy/Build UniRef90 Database

In [ ]:
# Install DIAMOND
!wget -q https://github.com/bbuchfink/diamond/releases/download/v2.1.8/diamond-linux64.tar.gz
!tar -xzf diamond-linux64.tar.gz
!mv diamond /usr/local/bin/

# Check if UniRef90 database exists from previous session
import os
UNIREF90_DB = f'{BASE_DIR}/databases/uniref90.dmnd'

if os.path.exists(UNIREF90_DB):
    print(f"Found existing UniRef90 database: {UNIREF90_DB}")
    !cp {UNIREF90_DB} .
else:
    print("UniRef90 database not found. Downloading (this will take ~40 minutes)...")
    !wget -q --show-progress https://ftp.uniprot.org/pub/databases/uniprot/uniref/uniref90/uniref90.fasta.gz
    !diamond makedb --in uniref90.fasta.gz -d uniref90 --threads {THREADS}
    !cp uniref90.dmnd {BASE_DIR}/databases/

## Step 5B: DIAMOND Search vs UniRef90

In [ ]:
# DIAMOND search against UniRef90
cmd = """diamond blastp \
    -q proteins.faa \
    -d uniref90 \
    -o uniref90_hits.m8 \
    --id 25 \
    --evalue 1e-5 \
    --sensitive \
    --threads 4 \
    --max-target-seqs 1"""

!{cmd}
log_command(cmd)

!echo "Proteins with UniRef90 hits:" && cut -f1 uniref90_hits.m8 | sort -u | wc -l

## Step 5C: SwissProt Search

In [ ]:
# Download and build SwissProt database
!wget -q https://ftp.ncbi.nlm.nih.gov/blast/db/FASTA/swissprot.gz
!gunzip -f swissprot.gz
!diamond makedb --in swissprot -d swissprot --quiet

# Search
cmd = """diamond blastp \
    -q proteins.faa \
    -d swissprot \
    -o swissprot_hits.m8 \
    --id 25 \
    --evalue 1e-5 \
    --sensitive \
    --threads 4"""

!{cmd}
log_command(cmd)

!echo "Proteins with SwissProt hits:" && cut -f1 swissprot_hits.m8 | sort -u | wc -l

## Step 5D: Human Contamination Check

In [ ]:
# Download human proteome
!wget -q https://ftp.uniprot.org/pub/databases/uniprot/current_release/knowledgebase/reference_proteomes/Eukaryota/UP000005640/UP000005640_9606.fasta.gz
!gunzip -f UP000005640_9606.fasta.gz
!diamond makedb --in UP000005640_9606.fasta -d human --quiet

# Search with relaxed thresholds
cmd = """diamond blastp \
    -q proteins.faa \
    -d human \
    -o human_hits.m8 \
    --id 50 \
    --evalue 1e-10 \
    --threads 4"""

!{cmd}
log_command(cmd)

human_count = !cut -f1 human_hits.m8 | sort -u | wc -l
print(f"Human contamination: {human_count[0]} proteins")
if int(human_count[0]) > 0:
    print("WARNING: Human contamination detected!")

## Step 5E: Chimera Detection (VSEARCH)

In [ ]:
# Install VSEARCH
!wget -q https://github.com/torognes/vsearch/releases/download/v2.22.1/vsearch-2.22.1-linux-x86_64.tar.gz
!tar -xzf vsearch-2.22.1-linux-x86_64.tar.gz
!cp vsearch-2.22.1-linux-x86_64/bin/vsearch /usr/local/bin/

# De novo chimera detection
cmd = """vsearch \
    --uchime_denovo genes.fna \
    --chimeras chimeras.fna \
    --nonchimeras clean_genes.fna \
    --quiet"""

!{cmd}
log_command(cmd)

!echo "Potential chimeras detected:" && grep -c ">" chimeras.fna || echo "0"

## Step 6: Extract Novel Proteins

In [ ]:
# Get all protein IDs
all_proteins = set()
with open('proteins.faa') as f:
    for line in f:
        if line.startswith('>'):
            all_proteins.add(line[1:].split()[0])

# Get proteins WITH hits
def get_hits(filename):
    hits = set()
    try:
        with open(filename) as f:
            for line in f:
                hits.add(line.split()[0])
    except FileNotFoundError:
        pass
    return hits

uniref90_hits = get_hits('uniref90_hits.m8')
swissprot_hits = get_hits('swissprot_hits.m8')
human_hits = get_hits('human_hits.m8')

# Novel = no hits anywhere
all_hits = uniref90_hits | swissprot_hits | human_hits
novel = all_proteins - all_hits

print(f"Total proteins: {len(all_proteins):,}")
print(f"UniRef90 hits: {len(uniref90_hits):,}")
print(f"SwissProt hits: {len(swissprot_hits):,}")
print(f"Human hits: {len(human_hits):,}")
print(f"NOVEL (no hits): {len(novel):,}")

# Extract novel sequences
seqs = {}
with open('proteins.faa') as f:
    h = None
    for line in f:
        if line.startswith('>'):
            h = line[1:].split()[0]
            seqs[h] = line
        else:
            seqs[h] += line

with open('novel_raw.faa', 'w') as f:
    for pid in novel:
        f.write(seqs[pid])

log_command(f"Extracted {len(novel)} novel proteins")

## Step 7: Quality Filtering

In [ ]:
# Load novel proteins
novels = []
with open('novel_raw.faa') as f:
    h, s = None, ''
    for line in f:
        if line.startswith('>'):
            if h: novels.append((h, s))
            h, s = line[1:].strip(), ''
        else:
            s += line.strip()
    if h: novels.append((h, s))

# Apply filters
filtered = []
rejected = {'short': 0, 'no_start': 0, 'internal_stop': 0, 'partial': 0}

for header, seq in novels:
    if len(seq) < 100:
        rejected['short'] += 1
    elif not seq.startswith('M'):
        rejected['no_start'] += 1
    elif '*' in seq[:-1]:
        rejected['internal_stop'] += 1
    elif 'partial=10' in header or 'partial=01' in header or 'partial=11' in header:
        rejected['partial'] += 1
    else:
        filtered.append((header, seq))

print(f"\nQUALITY FILTERING")
print(f"Input: {len(novels):,}")
print(f"Passed: {len(filtered):,}")
print(f"\nRejected:")
for reason, count in sorted(rejected.items(), key=lambda x: -x[1]):
    print(f"  {reason}: {count:,}")

# Save filtered
with open(f'{BASE_DIR}/output/truly_novel.faa', 'w') as f:
    for h, s in filtered:
        f.write(f'>{h}\n{s}\n')

log_command(f"Quality filtered: {len(novels)} -> {len(filtered)}")

## Step 8A: Amino Acid Composition Check

In [ ]:
from collections import Counter

EXPECTED = {'A': 0.083, 'R': 0.055, 'N': 0.041, 'D': 0.055, 'C': 0.014,
            'Q': 0.039, 'E': 0.068, 'G': 0.071, 'H': 0.023, 'I': 0.060,
            'L': 0.097, 'K': 0.058, 'M': 0.024, 'F': 0.039, 'P': 0.047,
            'S': 0.066, 'T': 0.053, 'W': 0.011, 'Y': 0.029, 'V': 0.069}

all_aa = ''.join(s for h, s in filtered)
counts = Counter(all_aa)
total = sum(counts[aa] for aa in EXPECTED.keys() if aa in counts)

chi_sq = 0
for aa, exp in EXPECTED.items():
    obs = counts.get(aa, 0) / total if total > 0 else 0
    chi_sq += ((obs - exp) ** 2) / exp

print(f"Chi-squared: {chi_sq:.3f}")
print("< 0.3 = normal protein composition")
print("> 0.5 = suspicious")
verdict_aa = "PASS" if chi_sq < 0.3 else "FAIL"
print(f"VERDICT: {verdict_aa}")

## Step 8B: Protein Classification

In [ ]:
def classify(seq):
    issues = []
    for aa in 'AGLPQRS':
        if aa * 5 in seq: issues.append('repetitive')
    if sum(1 for a in seq if a in 'AILMFWV') / len(seq) > 0.6:
        issues.append('too_hydrophobic')
    if sum(1 for a in seq if a in 'DEKR') / len(seq) > 0.4:
        issues.append('too_charged')
    return 'likely_real' if len(issues) == 0 else 'possibly_artifact'

real, artifact = 0, 0
for h, s in filtered:
    if classify(s) == 'likely_real':
        real += 1
    else:
        artifact += 1

print(f"Likely real: {real} ({real/len(filtered)*100:.1f}%)")
print(f"Possibly artifact: {artifact}")
verdict_class = "PASS" if real/len(filtered) > 0.9 else "REVIEW"
print(f"VERDICT: {verdict_class}")

## Step 8C: Genomic Context Analysis

In [ ]:
import re
from collections import defaultdict

# Parse GFF to get ORF positions
contig_orfs = defaultdict(list)
with open('genes.gff') as f:
    for line in f:
        if line.startswith('#') or not line.strip():
            continue
        parts = line.strip().split('\t')
        if len(parts) >= 9 and parts[2] == 'CDS':
            contig = parts[0]
            start, end = int(parts[3]), int(parts[4])
            pid_match = re.search(r'ID=([^;]+)', parts[8])
            if pid_match:
                pid = pid_match.group(1)
                contig_orfs[contig].append((start, end, pid))

# Find isolated ORFs
novel_ids = set(h.split()[0] for h, s in filtered)
isolated_novels = []

for contig, orfs in contig_orfs.items():
    for start, end, pid in orfs:
        if pid in novel_ids:
            if len(orfs) == 1:
                isolated_novels.append((pid, 'only_orf_on_contig'))
            elif max(e for s, e, p in orfs) - min(s for s, e, p in orfs) < 500:
                isolated_novels.append((pid, 'very_short_contig'))

print(f"Isolated novel ORFs (lower confidence): {len(isolated_novels)}")
print(f"Contextual novel ORFs (higher confidence): {len(novel_ids) - len(isolated_novels)}")
verdict_context = "REVIEW" if len(isolated_novels) > len(novel_ids) * 0.3 else "PASS"
print(f"VERDICT: {verdict_context}")

## Step 8D: HHblits Sample Generation (Manual Step)

In [ ]:
import random
random.seed(42)
sample_20 = random.sample(filtered, min(20, len(filtered)))

print("="*60)
print("HHblits REMOTE HOMOLOGY TEST (Manual Step)")
print("="*60)
print("\nInstructions:")
print("1. Go to: https://toolkit.tuebingen.mpg.de/tools/hhpred")
print("2. Select database: PDB_mmCIF70, UniRef30")
print("3. Paste each sequence below, one at a time")
print("4. Record: Match found (Type C) or No match (potentially Type A)")
print("\n")

for i, (h, s) in enumerate(sample_20, 1):
    print(f"=== Sample {i}/20 ===")
    print(f"ID: {h.split()[0]}")
    print(f"Length: {len(s)} aa")
    print(f"Sequence:\n{s[:200]}{'...' if len(s) > 200 else ''}")
    print()

## Step 8E: Foldseek Sample Generation (Manual Step)

In [ ]:
sample_10 = random.sample(filtered, min(10, len(filtered)))

print("="*60)
print("FOLDSEEK STRUCTURAL SEARCH (Manual Step)")
print("="*60)
print("\nInstructions:")
print("1. Go to: https://search.foldseek.com/search")
print("2. For each sequence below:")
print("   a. First predict structure at: https://esmatlas.com/resources?action=fold")
print("   b. Download the PDB file")
print("   c. Upload to Foldseek, search against PDB, AlphaFold")
print("3. Record: Structure match (Type B) or No match (potentially Type A)")
print("\n")

for i, (h, s) in enumerate(sample_10, 1):
    print(f"=== Sample {i}/10 ===")
    print(f"ID: {h.split()[0]}")
    print(f"Length: {len(s)} aa")
    if len(s) > 400:
        print("WARNING: >400aa, may need to truncate for ESMFold")
    print(f"Sequence:\n{s[:300]}{'...' if len(s) > 300 else ''}")
    print()

## Step 8F: Generate Verification Report

In [ ]:
# Classification
classifications = {
    'type_a_completely_novel': [],
    'type_b_structure_known': [],
    'type_c_remote_homolog': [],
    'type_d_domain_hybrid': [],
    'type_e_artifact': []
}

# Add artifacts
artifact_ids = set()
for pid, reason in isolated_novels:
    classifications['type_e_artifact'].append((pid, reason))
    artifact_ids.add(pid)

# Remaining as Type A (preliminary)
for h, s in filtered:
    pid = h.split()[0]
    if pid not in artifact_ids:
        classifications['type_a_completely_novel'].append(pid)

# Generate report
report = f"""# Verification Report - Silent Hunter v6.0

Generated: {datetime.datetime.now().isoformat()}

## Summary

| Metric | Value |
|--------|-------|
| Total proteins predicted | {len(all_proteins):,} |
| Novel (no database hits) | {len(novel):,} |
| Quality filtered | {len(filtered):,} |

## Database Searches

| Database | Size | Hits | Novel |
|----------|------|------|-------|
| UniRef90 | 184M | {len(uniref90_hits):,} | {len(all_proteins)-len(uniref90_hits):,} |
| SwissProt | 570K | {len(swissprot_hits):,} | - |
| Human | 20K | {len(human_hits):,} | - |

## Verification Tests

| Test | Result | Verdict |
|------|--------|--------|
| Human contamination | {len(human_hits)} matches | {"PASS" if len(human_hits) == 0 else "FAIL"} |
| AA composition | Chi-sq = {chi_sq:.3f} | {verdict_aa} |
| Protein classification | {real/len(filtered)*100:.1f}% real | {verdict_class} |
| Genomic context | {len(isolated_novels)} isolated | {verdict_context} |

## Novel Protein Classification (Preliminary)

| Type | Description | Count |
|------|-------------|-------|
| Type A | Completely novel | {len(classifications['type_a_completely_novel'])} |
| Type B | Structure-known | TBD (run Foldseek) |
| Type C | Remote homolog | TBD (run HHblits) |
| Type D | Domain hybrid | TBD (run Pfam) |
| Type E | Possible artifact | {len(classifications['type_e_artifact'])} |

## Manual Verification Required

- [ ] HHblits remote homology (20 samples)
- [ ] Foldseek structural similarity (10 samples)
- [ ] NCBI nr spot check (10 samples)
- [ ] Pfam domain search (full dataset)

## Conclusion

**{len(filtered):,} novel protein candidates** identified from ISS metagenome.

After full verification, expect:
- **Type A (Truly Novel):** ~{int(len(filtered)*0.15)}-{int(len(filtered)*0.25)} proteins
- **Type B-D (Classifiable):** ~{int(len(filtered)*0.6)}-{int(len(filtered)*0.7)} proteins
- **Type E (Artifacts):** ~{int(len(filtered)*0.05)}-{int(len(filtered)*0.1)} proteins
"""

with open(f'{BASE_DIR}/output/VERIFICATION_REPORT.md', 'w') as f:
    f.write(report)

print(report)

## Save Checkpoints

In [ ]:
# Save all intermediate files to Drive
!mkdir -p {BASE_DIR}/intermediate/diamond
!cp *_hits.m8 {BASE_DIR}/intermediate/diamond/ 2>/dev/null || true

# Calculate final checksums
!md5sum {BASE_DIR}/output/truly_novel.faa >> {BASE_DIR}/audit/checksums.md5

print(f"\nAll files saved to: {BASE_DIR}")
print("\nFinal output:")
!ls -lh {BASE_DIR}/output/

## Pipeline Complete!

### Next Steps:

1. **Manual HHblits verification** - Use the samples from Step 8D
2. **Manual Foldseek verification** - Use the samples from Step 8E
3. **NCBI nr spot check** - Run 10 random proteins through web BLAST
4. **Update classification** - Based on manual results
5. **Prepare publication** - Use METHODS.md as starting point